In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [2]:
# from sql.MongoDBArticle import MongoDBArticle

In [3]:
# MongoDBArticle.get_collection()

In [4]:
# a = MongoDBArticle.fetch_all_document_ids()

In [5]:
# SSH tunnel configuration
SSH_HOST=''
SSH_USER='ec2-user'
SSH_KEY_PATH=''
LOCAL_BIND_PORT=3000

# MongoDB server configuration
MONGO_HOST='insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com'
MONGO_PORT=27017
MONGO_USERNAME=''
MONGO_PASSWORD=''


MONGO_DB_NAME='insight_db'
MONGO_COLLECTION_NAME='articles'

# test environment
ENVIRONMENT='local'

In [6]:
DB_USERNAME = MONGO_USERNAME
DB_PASSWORD=MONGO_PASSWORD

In [7]:
import os

In [8]:

import os
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder

# # SSH tunnel configuration
# SSH_HOST = os.environ.get('SSH_HOST')
# SSH_USER = os.environ.get('SSH_USER')
# SSH_KEY_PATH = os.environ.get('SSH_KEY_PATH')
# LOCAL_BIND_PORT = int(os.environ.get('LOCAL_BIND_PORT', 3010))


# # MongoDB server configuration
# MONGO_HOST = os.environ.get('MONGO_HOST')
# MONGO_PORT = int(os.environ.get('MONGO_PORT', 27017))
# DB_USERNAME = os.environ.get('MONGO_USERNAME')
# DB_PASSWORD = os.environ.get('MONGO_PASSWORD')

# db parameters dict
DB_PARAMS = {
    "host": '127.0.0.1',
    "port": LOCAL_BIND_PORT,
    "username": DB_USERNAME,
    "password": DB_PASSWORD,
}


class MongoDatabaseConnection:
    _instance = None
    _client = None
    _tunnel = None

    def __new__(cls):
        if cls._instance is None:
            try:
                cls._instance = super(MongoDatabaseConnection, cls).__new__(cls)

                if os.environ.get('ENVIRONMENT') == 'local':
                    # initiate mongo client via ssh tunneling
                    cls._instance._client = cls._connect_to_mongodb_using_ssh()
                else:
                    # initiate mongo client
                    cls._instance._client = cls._connect_to_mongodb()
            except Exception as e:
                raise Exception(f"Failed to connect to MongoDB: {e}")

        return cls._instance

    @classmethod
    def get_client(cls):
        return cls._instance._client

    @classmethod
    def close_connection(cls):
        if cls._instance._client:
            cls._instance._client.close()
            cls._instance._client = None

        if cls._tunnel:
            cls._tunnel.stop()
            cls._tunnel = None

    @classmethod
    def _establish_tunnel(cls):
        try:
            tunnel = SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username=SSH_USER,
                ssh_pkey=SSH_KEY_PATH,
                remote_bind_address=(MONGO_HOST, MONGO_PORT),
                local_bind_address=('127.0.0.1', LOCAL_BIND_PORT)
            )

            return tunnel
        except Exception as e:
            raise Exception(f"Failed to establish SSH tunnel: {e}")

    @classmethod
    def _connect_to_mongodb(cls):
        try:
            return MongoClient(**DB_PARAMS)
        except ConnectionError as mongo_error:
            raise Exception(f"Failed to connect to MongoDB Connection: {mongo_error}")

    @classmethod
    def _connect_to_mongodb_using_ssh(cls):
        try:
            # start the tunnel
            cls._tunnel = cls._establish_tunnel()
            cls._tunnel.start()

            return MongoClient(
                directConnection=True,
                **DB_PARAMS
            )
        except ConnectionError as mongo_error:
            raise Exception(f"Failed to connect to MongoDB Connection: {mongo_error}")

In [9]:
from bson import ObjectId

In [10]:
class MongoDBArticle:

    @staticmethod
    def get_db():
        mongo_conn = None
        try:
            # setup mongo connection
            mongo_conn = MongoDatabaseConnection()
            mongo_client = mongo_conn.get_client()
            db = mongo_client[MONGO_DB_NAME]
            return db
        except Exception as e:
            if mongo_conn:
                mongo_conn.close_connection()
            raise Exception(f"Error in connecting to db: {e}")

    @staticmethod
    def get_collection(collection_name=None):
        try:
            db = MongoDBArticle.get_db()
            if collection_name is None:
                return db[MONGO_COLLECTION_NAME]
            else:
                return db[collection_name]
        except Exception as e:
            raise Exception(f"Error in _get_collection: {e}")

    @staticmethod
    def fetch_documents_by_ids(string_ids):
        try:
            object_ids = [ObjectId(string_id) for string_id in string_ids]
        except Exception as e:
            return f"Invalid ID format: {e}"

        collection = MongoDBArticle.get_collection()
        documents = list(collection.find({"_id": {"$in": object_ids}}))
        doc_dict = {}
        for doc in documents:
            article_id = str(doc.pop('_id'))
            doc_dict[article_id] = doc
            doc_dict[article_id]['article_id'] = article_id
            doc_dict[article_id]['source_id'] = str(doc['source_id'])
            doc_dict[article_id]['is_premium_article'] = False
        return doc_dict
    
    @staticmethod
    def fetch_all_document_ids():
        query = {"is_premium_article": False}
        projection = {"_id": 1}

        collection = MongoDBArticle.get_collection()
        documents = list(collection.find(query, projection).limit(100))
        return [str(doc['_id']) for doc in documents]

In [11]:
from requests_aws4auth import AWS4Auth
import os
import requests
import json
import pandas as pd

In [12]:
ES_SERVICE = 'aoss'
ES_AUTH = AWS4Auth(os.environ.get('ES_ACCESS_KEY'), os.environ.get('ES_SECRET_KEY'), 'ap-southeast-1', ES_SERVICE)
ES_URL = f'https://{os.environ.get("ES_HOST")}/finance-contents/_search'

In [13]:
def perform_exact_search(query_text, n=100):
    headers = {"Content-Type": "application/json"}
    query = {
        "query": {
            "match": {
                "cleaned_text": {
                    "query": query_text
                }
            }
        }
    }
    response = requests.get(ES_URL, auth=ES_AUTH, headers=headers, json=query)
    results = json.loads(response.text)['hits']['hits'][:n]
    return pd.DataFrame([(x['_id'], x['_score']) for x in results], columns=['article_id', 'score'])

In [14]:
# df = perform_exact_search('india')

In [15]:
# df

In [16]:
article_ids = MongoDBArticle.fetch_all_document_ids()

In [17]:
ids = ['659bd5c24b25ce4814fb5159',
'659bd5c24b25ce4814fb5155',
'659bd5c24b25ce4814fb5154',
'659bd5c44b25ce4814fb515d',
'659bd5c24b25ce4814fb5158',
'659bd5c14b25ce4814fb5151',
'659b9d874b25ce4814fb50ff',
'659bd5c54b25ce4814fb515e',
'659bd5c34b25ce4814fb515c',
'656ddb3b9994965ac4b450ca']

In [18]:
article_docs = MongoDBArticle.fetch_documents_by_ids(ids)

In [19]:
article_docs

{'656ddb3b9994965ac4b450ca': {'url': 'https://www.cnbc.com/2023/11/30/lyft-ceo-touts-success-of-airport-on-time-pickup-promise-for-riders.html',
  'source_id': '6555995596fb32eef21a562e',
  '_class': 'com.wintWealth.insight.common.models.entities.Article',
  'image_url': 'https://image.cnbcfm.com/api/v1/image/105770193-1551467099728preview.jpg?v=1701371905&w=1920&h=1080',
  'is_premium_article': False,
  'last_updated_time': '2023-12-01T00:11:54+00:00',
  'published_time': '2023-12-01T00:10:55+00:00',
  'short_description': 'Lyft CEO David Risher told CNBC\'s Jim Cramer about the success of his company\'s new "on-time pickup promise" launched ahead of the holiday travel season.',
  'tags': ['Uber Technologies Inc,Lyft Inc,Markets,Business,Investment strategy,Stock markets,Jim Cramer,business news'],
  'title': "Lyft CEO touts success of airport 'on-time pickup promise' for riders",
  'authors': ['Julie Coleman'],
  'ai_generated_info': {'summary': {'OpenHermes_WatermelonSapphireZipline

In [20]:
from src.Article import Article

In [21]:
articles = {}
for id, article in article_docs.items():
    articles[id] = Article.from_dict(article)

In [22]:
articles['656ddb3b9994965ac4b450ca'].cleaned_text

'watch now VIDEO 1:12 01:12 Lyft CEO David Risher talks the company\'s on-time pickup promise Mad Money with Jim Cramer Lyft CEO David Risher told CNBC\'s Jim Cramer on Thursday said he\'s bullish on the company\'s " on-time pickup promise " program, which was launched before the busy holiday travel season. "Everyone is stressed during holidays, and the holidays should be so great — they\'re a time of getting together with your family — but man, what time am I getting to the airport? Is a car going to pick me up?" Risher said. "We guarantee we\'re going to pick you up. If we are more than ten minutes late, we will pay you up to $100 in Lyft cash." Risher said over the past few weeks, Lyft has given 2.5 million rides to the airport and only had to reimburse 72 rides for late pick-ups. Lyft has struggled in the ride-share space since its 2019 initial public offering, with competitor Uber pulling ahead in terms of market share and market capitalization. Risher reported steady improvement 

In [23]:
articles['656ddb3b9994965ac4b450ca'].full_content

'headline: Lyft CEO touts success of airport on-time pickup promise for riders.  content: watch now VIDEO 1:12 01:12 Lyft CEO David Risher talks the companys on-time pickup promise Mad Money with Jim Cramer Lyft CEO David Risher told CNBCs Jim Cramer on Thursday said hes bullish on the companys  on-time pickup promise  program which was launched before the busy holiday travel season. Everyone is stressed during holidays and the holidays should be so great  theyre a time of getting together with your family  but man what time am I getting to the airport Is a car going to pick me up Risher said. We guarantee were going to pick you up. If we are more than ten minutes late we will pay you up to $100 in Lyft cash. Risher said over the past few weeks Lyft has given 2.5 million rides to the airport and only had to reimburse 72 rides for late pick-ups. Lyft has struggled in the ride-share space since its 2019 initial public offering with competitor Uber pulling ahead in terms of market share a

In [24]:
from src.RankingService import RankingService

In [27]:
a = RankingService.get_reranked_search_results(query_text='india', articles=list(articles.values()), num_results=100)

fetched results from cohere in 0:00:00.683846 second


In [30]:
articles['659bd5c14b25ce4814fb5151'].full_content

'headline: Thali costs fall in December  Mint.  content:  New Delhi: The cost of home-cooked vegetarian and non-vegetarian meals or thali fell sequentially in December following the end of the festival season. The price of a vegetarian thali fell 3% to ₹ 29.7 last month while non-vegetarian thali saw a 5% decline to ₹ 57.6 according to rating agency Crisil. Year-on-year the cost of a vegetarian thali increased whereas non-vegetarian thali became cheaper. A typical vegetarian thali includes roti vegetables (onion tomato potato) rice dal curd and salad. The non-vegetarian version replaces dal with chicken (broiler). Despite fluctuating commodity prices the share of ingredients used for arriving at the prices remains constant. Crisil attributed the drop in the vegetarian thalis cost to a 14% and 3% decrease in onion and tomato prices respectively as festive demand waned. The more significant decline in the non-vegetarian thalis cost was driven by a 5-7% reduction in broiler prices which c

In [28]:
a

{'searchArticleIds': [{'article_id': '659bd5c54b25ce4814fb515e'},
  {'article_id': '659bd5c14b25ce4814fb5151'},
  {'article_id': '659bd5c24b25ce4814fb5154'},
  {'article_id': '659bd5c34b25ce4814fb515c'},
  {'article_id': '659bd5c24b25ce4814fb5155'},
  {'article_id': '659bd5c24b25ce4814fb5159'},
  {'article_id': '659b9d874b25ce4814fb50ff'},
  {'article_id': '659bd5c24b25ce4814fb5158'},
  {'article_id': '659bd5c44b25ce4814fb515d'},
  {'article_id': '656ddb3b9994965ac4b450ca'}],
 'additionalInfo': {'659bd5c54b25ce4814fb515e': {'published_time': Timestamp('2024-01-08 09:50:06+0000', tz='UTC'),
   'num_days_ago': 13,
   'recency_score': 0.708,
   'relevance_score': 0.8912444,
   'weighted_score': 0.8454333},
  '659bd5c14b25ce4814fb5151': {'published_time': Timestamp('2024-01-08 10:08:55+0000', tz='UTC'),
   'num_days_ago': 13,
   'recency_score': 0.708,
   'relevance_score': 0.88759494,
   'weighted_score': 0.842696205},
  '659bd5c24b25ce4814fb5154': {'published_time': Timestamp('2024-01-08